<a href="https://colab.research.google.com/github/kyanite153/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#大富豪（トランプゲーム）

#ソースコード

In [ ]:
import pandas as pd
import random

class CareerPoker:
  def __init__(self, name='player1'):
    self.player_name = name
    self.games = 1
    self.number_of_players = 4
    self.istrump_8 = True
    self.isreg_suit = True
  def is_play_card(self, player, cards=None):
    if cards == None:
      l = []
      for a in self.state[player]:
        if self.is_play_card(player, {a}):
          l.append({a})
        for b in self.state[player]:
          if a == b:
            continue
          if self.is_play_card(player, {a, b}):
            l.append({a, b})
          for c in self.state[player]:
            if len({a,b,c}) != 3:
              continue
            if self.is_play_card(player, {a, b, c}):
              l.append({a, b, c})
            for d in self.state[player]:
              if len({a,b,c,d}) != 4:
                continue
              if self.is_play_card(player, {a, b, c, d}):
                l.append({a, b, c, d})
      return l
    if not cards <= self.state[player]:
      #場に出そうとしているカードが手札にない
      return False
    if 'joker' in cards and len(cards)>=2:
      #備考：５枚以上は現状不可．
      new_cards = set(cards)
      new_cards.remove('joker')
      for x in self.get_all_card():
        if x == 'joker' or x in cards:
          continue
        new_cards.add(x)
        if self.is_play_card(player, new_cards):
          return True
        new_cards.remove(x)
    if len({x[1:] for x in cards}) == 1:
      #1枚以上の同じ数字のカード
      if len(self.cards_select)==0:
        return True
      elif len(cards) == len(self.cards_select):
        if self.get_card_strength(list(cards)[0]) > self.get_card_strength(list(self.cards_select)[0]):
          if self.reg_suit != None and len(cards) == 1 and cards != {'joker'} and(list(cards)[0][0] != self.reg_suit):
            return False
          return True
    #現時点では階段には非対応(jokerを含む場合の処理が難しい)．
    '''
    a = [self.get_card_strength(x) for x in list(cards)]
    b = [a[x+1]-a[x] for x in range(len(a)-1)]
    if set(b)=={1}:
      return True
    '''
    return False
  def sort_cards(self, cards):
    l = []
    for x in self.card_strength:
      for card in cards:
        if card[1:] == x:
          l.append(card)
    return l
  def set_game(self):
    while True:
      print('確認/変更したいものを選択．')
      print('1:ラウンド数')
      print('2:プレイ人数')
      print('3:8切り')
      print('4:記号しばり')
      print('5:設定を終了する')
      n = int(input('ここに数字を入力:'))
      if n == 1:
        print(f'現在の設定：{self.games}')
        n = input('ラウンド数：')
        if 1<=n:
          self.games = n
        else:
          print('エラー．ラウンド数は1以上．')
      elif n == 2:
        print(f'現在の設定：{self.number_of_players}')
        n = input('プレイ人数：')
        if 3<=n:
          self.number_of_players = n
        else:
          print('エラー．プレイ人数は3以上．')
      elif n == 3:
        print(f'現在の設定：{self.istrump_8}')
        print('0:無効')
        print('1:有効')
        n = int(input('ここに数字を入力:'))
        if n == 0:
          self.istrump_8 = False
        elif n == 1:
          self.istrump_8 = True
        else:
          print('エラー')
      elif n == 4:
        print(f'現在の設定：{self.isreg_suit}')
        print('0:無効')
        print('1:有効')
        n = int(input('ここに数字を入力:'))
        if n == 0:
          self.isreg_suit = False
        elif n == 1:
          self.isreg_suit = True
        else:
          print('エラー')
      elif n == 5:
        #設定終了
        break
  def get_card_strength(self, card):
    return self.card_strength.index(card[1:])
  def get_all_card(self):
    l = ['3','4','5','6','7','8','9','10','J','Q','K','A','2']
    return {f'h{x}' for x in l}|{f'd{x}' for x in l}|{f's{x}' for x in l}|{f'c{x}' for x in l}|{'joker'}
  def run_game(self):
    com = [CareerPokerCOM(self, f'com{x+1}') for x in range(self.number_of_players-1)]
    players = [self.player_name]+com
    score_data = pd.DataFrame([[players[j]]+[None for i in range(self.games)] for j in range(self.number_of_players)], columns=['name']+[f'{x}試合目' for x in range(1,self.games+1)])
    display(score_data)
    self.reg_suit = None
    all_card = self.get_all_card()
    self.state = {players[i]:set() for i in range(self.number_of_players)}
    self.state['deck'] = all_card
    p_list = list(players)
    while len(self.state['deck']) != 0:
      if len(p_list) == 0:
        p_list = list(players)
      c = random.choice(list(self.state['deck']))
      p = random.choice(p_list)
      self.state[p].add(c)
      p_list.remove(p)
      self.state['deck'].remove(c)
    self.card_strength = ['3','4','5','6','7','8','9','10','J','Q','K','A','2','oker']
    self.cards_select = set()
    ranking = []
    round = 1
    if round == self.games:
      print(f'ファイナルラウンド')
    else:
      print(f'ラウンド{round}')
    #始めの親を決める
    parent = random.choice(players)
    print(f'最初の親は {parent} です．')
    count_pass = 0
    while True:
      #ゲーム開始
      ans = {'pass'}
      if parent == self.player_name:
        print(f'{parent} の番です．')
        print('(手札)',end=' ')
        [print(f'{x}:{len(self.state[x])}枚', end=' ') for x in players]
        print()
        if self.reg_suit != None:
          print(f'スートしばり({self.reg_suit})')
        print(f'場のカード：{list(self.cards_select)}')
        print(f"自分の手札：{self.sort_cards(list(self.state[parent]))}")
        while True:
          ans = set(input('場に出すカード：').split())
          if ans != {'pass'}:
            count_pass = 0
            bool1 = master.is_play_card(parent, ans)
            if bool1:
              self.state[parent] -= ans
              if self.isreg_suit and self.reg_suit==None and len(self.cards_select)==1 and list(ans)[0][0] == list(self.cards_select)[0][0]:
                self.reg_suit = list(ans)[0][0]
                print('スートしばり')
              self.cards_select = ans
              break
            else:
              print('そのカードは出せません．(「pass」と入力するとパス．)')
          else:
            count_pass += 1
            break
      else:
        #com
        ans = parent.play_card()
        if ans != {'pass'}:
          count_pass = 0
          print(f'{parent}が出したカード:{ans}')
          if not master.is_play_card(parent, ans):
            print('ERROR!')
          self.state[parent] -= ans
          if self.isreg_suit and self.reg_suit==None and len(self.cards_select)==1 and list(ans)[0][0] == list(self.cards_select)[0][0]:
            self.reg_suit = list(ans)[0][0]
            print('スートしばり')
          self.cards_select = ans
        else:
          count_pass += 1
          print(f'{parent}:pass')
      if len(ans) == 4:
        print('\n革命！！\n')
        self.card_strength.remove('oker')
        self.card_strength.reverse()
        self.card_strength.append('oker')
      if (self.istrump_8 and list(ans)[0][1:]=='8') or count_pass >= len(players)-1:
        if (self.istrump_8 and list(ans)[0][1:]=='8'):
          print('8切り')
          parent = players[(players.index(parent)-1)%len(players)]
        print('場が流れます．')
        self.reg_suit = None
        self.cards_select = set()
        count_pass = 0
      if len(self.state[parent]) == 0:
        #手札が0枚の時
        print(f'{parent}:あがり！')
        ranking.append(parent)
        parent = players[(players.index(parent)+1)%len(players)]
        players.remove(ranking[-1])
      else:
        #手札が残っているとき．
        parent = players[(players.index(parent)+1)%len(players)]
      if len(ranking)+1 == self.number_of_players:
        #ラウンド終了
        ranking.append(parent)
        for player in ranking:
          score_data[f'{round}試合目'][score_data['name']==player] = (len(ranking)-ranking.index(player))*10
        if round == self.games:
          break
        #次のラウンドへの準備
        round += 1
        players = [self.player_name]+com
        display(score_data)
        self.reg_suit = None
        self.state = {players[i]:set() for i in range(self.number_of_players)}
        all_card = self.get_all_card()
        self.state['deck'] = all_card
        p_list = list(players)
        while len(self.state['deck']) != 0:
          if len(p_list) == 0:
            p_list = list(players)
          c = random.choice(list(self.state['deck']))
          p = random.choice(p_list)
          self.state[p].add(c)
          p_list.remove(p)
          self.state['deck'].remove(c)
        self.card_strength = ['3','4','5','6','7','8','9','10','J','Q','K','A','2','oker']
        self.cards_select = set()
        ranking = []
        if round == self.games:
          print(f'ファイナルラウンド')
        else:
          print(f'ラウンド{round}')
        #始めの親を決める
        parent = random.choice(players)
        print(f'最初の親は {parent} です．')
    #ゲームセット
    print('ゲーム終了')
    score_data['合計'] = sum([score_data[f'{i+1}試合目'] for i in range(self.games)])
    score_data = score_data.sort_values('合計', ascending=False)
    display(score_data)
    print(f'あなたの順位：{list(score_data["name"]).index(self.player_name)+1}位')

class CareerPokerCOM:
  def __init__(self, master, name='com'):
    self.strength = 1 #1~3
    self.master = master
    self.name = name
  def play_card(self):
    cards_pattern = master.is_play_card(self)
    if len(cards_pattern) == 0:
      return {'pass'}
    else:
      return random.choice(cards_pattern)
  def __repr__(self):
    return self.name

def main():
  global master
  player_name = 'player1'
  while True:
    player_name = input('あなたの名前を入力：')
    if player_name[:3] == 'com':
      print('その名前は使用できません')
    else:
      break
  master=CareerPoker(player_name)
  while True:
    print('以下から数字で選択．')
    print('1:ゲーム開始')
    print('2:設定')
    print('3:終了')
    n = int(input('ここに数字を入力:'))
    if n == 1:
      master=CareerPoker(player_name)
      print('ゲーム開始')
      master.run_game()
    elif n == 2:
      master.set_game()
    elif n == 3:
      print('Thank you for playing!')
      break

if __name__ == '__main__':
  main()
